In [1]:
import json, re,  os, time, random, time
from bs4 import BeautifulSoup   
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC  
from config import *
from datetime import datetime, timedelta

<a id="id"></a>

In [2]:
config=''
with open(r'C:\Users\timko\source\repos\PythonBoiler\CodeGenerators\config.json') as f:
    config = json.loads(f.read())  


In [4]:
options = Options()
options.add_argument("--start-minimized")  
options.add_argument("--window-size=1220,980")  
driver = webdriver.Chrome(executable_path=ChromeDriverManager(log_level=0, version='116.0.5845.96').install() ,options=options)   
driver.get(f"https://dayman.cyber-balance.com/jira/login.jsp")  
driver.find_element(By.XPATH, '//input[contains(@id, "user")]').send_keys(config['user'])
driver.find_element(By.XPATH, '//input[contains(@id, "pass")]').send_keys(config['pass']) 
driver.find_element(By.XPATH, '//input[contains(@id, "submit")]').click() 
driver.get(f'https://dayman.cyber-balance.com/jira/secure/Dashboard.jspa')
eles = driver.find_elements_by_xpath("//a[contains(text(),'BOD 23')]")
issues=[e.get_attribute("href") for e in eles]




====== WebDriver manager ======
Current google-chrome version is 116.0.5845
There is no [win32] chromedriver for browser 116.0.5845 in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/115.0.5735.90/chromedriver_win32.zip


ValueError: There is no such driver by url https://chromedriver.storage.googleapis.com/115.0.5735.90/chromedriver_win32.zip

In [ ]:

for issue in issues[:]: 
    print(issue) 
    driver.get(issue)  
    try:
        ele = WebDriverWait(driver, timeout=2).until(lambda d: d.find_elements_by_xpath("//span[contains(text(),'Start Progress')]")) 
        if len(ele) > 0:
            ele[0].click() 
            driver.get(issue) 
            time.sleep(1)
    except Exception as e:
        print(e) 
    ele = WebDriverWait(driver, timeout=2).until(lambda d: d.find_element_by_xpath("//a[@id='opsbar-operations_more']"))
    ele.click()
    ele = WebDriverWait(driver, timeout=2).until(lambda d: d.find_element_by_xpath("//span[contains(text(),'Log work')]"))
    ele.click()
    ele = WebDriverWait(driver, timeout=2).until(lambda d: d.find_element_by_xpath("//*[@id='log-work-time-logged']"))
    ele.send_keys('20m') 
    ele = driver.find_elements_by_xpath("//*[@id='log-work-submit']")
    ele[0].click()
    time.sleep(5)




In [ ]:
for issue in issues[:]:  
    driver.get(issue)   
    time.sleep(5)  
    print(issue)   
    try:
        ele = WebDriverWait(driver, timeout=5).until(lambda d: d.find_elements_by_xpath("//span[contains(text(),'Resolve Issue')]")) 
        if len(ele) > 0:
            ele[0].click()   
            ele = WebDriverWait(driver, timeout=2).until(lambda d: d.find_element_by_xpath("//input[@id='issue-workflow-transition-submit']")) 
            ele.click()   
    except Exception as e:
        print(e) 

    try:
        ele = WebDriverWait(driver, timeout=5).until(lambda d: d.find_elements_by_xpath("//span[contains(text(),'Ready To Test')]")) 
        if len(ele) > 0:
            ele[0].click()    
            ele = WebDriverWait(driver, timeout=2).until(lambda d: d.find_element_by_xpath("//input[@id='issue-workflow-transition-submit']")) 
            ele.click()   
    except Exception as e:
        print(e) 